In [1]:
import simpletransformers 
import pandas as pd
from collections import defaultdict
from pathlib import Path
import os
import csv
import pandas as pd
main_folder = 'Data/MD'

In [2]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
from pycm import ConfusionMatrix

import os
import sys

In [3]:
results = defaultdict(list)
for file in Path(main_folder).iterdir():
    with open(file, encoding = "ISO-8859-1") as f:
        #lines = [x.decode('utf8').strip() for x in f.readlines()]
        results["file_name"].append(file.name)
        results["Text"].append(f.read())
        
df = pd.DataFrame(results)

In [4]:
results = defaultdict(list)
for file in Path('Data/NoMD').iterdir():
    with open(file, encoding = "ISO-8859-1") as file_open:
        results["file_name"].append(file.name)
        results["Text"].append(file_open.read())
        
df2 = pd.DataFrame(results)

In [5]:
df2['Disorder']='nomd'
df['Disorder']='md'
result1 = pd.concat([df, df2])
del result1['file_name']

In [ ]:
dfmentald = pd.read_excel(r'MentalDisorders/LIWCresultstotal.xlsx')
df.columns = ['Filename','Text']
result = pd.merge(df, dfmentald, on='Filename', how = 'outer').fillna(0)
del result['Filename']
result1 = result

In [7]:
result1['Disorder'] = pd.factorize(result1['Disorder'])[0]

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('dutch')

In [ ]:
result1['Text'] = result1['Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [ ]:
from sklearn.model_selection import train_test_split
# Train Test Split Function
def split_train_test(result, test_size=0.2, shuffle_state=True):
    X_train, X_test, Y_train, Y_test = train_test_split(result1[['Text']], 
                                                       result1['Disorder'], 
                                                        shuffle=shuffle_state,
                                                        test_size=test_size, 
                                                        random_state=15)
    print("Value counts for Train sentiments")
    print(Y_train.value_counts())
    print("Value counts for Test sentiments")
    print(Y_test.value_counts())
    print(type(X_train))
    print(type(Y_train))
    X_train = X_train.reset_index()
    X_test = X_test.reset_index()
    Y_train = Y_train.to_frame()
    Y_train = Y_train.reset_index()
    Y_test = Y_test.to_frame()
    Y_test = Y_test.reset_index()
    print(X_train.head())
    return X_train, X_test, Y_train, Y_test

# Call the train_test_split
X_train, X_test, Y_train, Y_test = split_train_test(result1)

In [9]:
result2 = pd.merge(Y_train, X_train, on='index', how = 'outer').fillna(0)
del result2['index']

In [10]:
result3 = pd.merge(Y_test, X_test, on='index', how = 'outer').fillna(0)
del result3['index']

In [ ]:
#prefix = 'MentalDisorders/'

train_df = result2
train_df.head()

eval_df = result3
eval_df.head()

train_df['Disorder'] = pd.to_numeric(train_df['Disorder'])
eval_df['Disorder'] = pd.to_numeric(eval_df['Disorder'])

train_df = pd.DataFrame({
    'text': train_df["Text"].replace(r'\n', ' ', regex=True),
    'label':train_df['Disorder']
})

print(train_df.head())

eval_df = pd.DataFrame({
    'text': eval_df['Text'].replace(r'\n', ' ', regex=True),
    'label':eval_df['Disorder']
})

print(eval_df.head())

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

model_args = ClassificationArgs(num_train_epochs= 3,  overwrite_output_dir= True, dataloader_num_workers = 0)

# Create a TransformerModel
model = ClassificationModel('roberta', 'pdelobelle/robbert-v2-dutch-base', use_cuda=False, num_labels = 2,
                            args = model_args)
#
# Train the model
model.train_model(train_df)

# Evaluate the model
#result, model_outputs, wrong_predictions = model.eval_model(eval_df)

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

In [ ]:
print(result, model_outputs, wrong_predictions)

In [ ]:
import numpy as np
import sklearn
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
lst = []
for arr in model_outputs:
    lst.append(np.argmax(arr))
true = eval_df['label'].tolist()
predicted = lst
sklearn.metrics.accuracy_score(true,predicted)

In [ ]:
import numpy as np
import sklearn
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, cohen_kappa_score
lst = []
for arr in model_outputs:
    lst.append(np.argmax(arr))
true = eval_df['label'].tolist()
predicted = lst
sklearn.metrics.cohen_kappa_score(true, predicted)

In [16]:
import lime.lime_text
import re
import webbrowser
import numpy as np


def strip_formatting(string):
    string = string.lower()
    string = re.sub(r"([.!?,'/()])", r" \1 ", string)
    return string

def tokenize_string(string):
    return string.split()

classifier = (model)


explainer = lime.lime_text.LimeTextExplainer(
    split_expression=tokenize_string,
    bow=False,
    class_names=["0", "1"]
)

def fasttext_prediction_in_sklearn_format(classifier, texts):
    res = []
    labels, probabilities = classifier.predict(texts, 10)
    for label, probs, text in zip(labels, probabilities, texts):
        order = np.argsort(np.array(label))
        res.append(probs[order])

    return np.array(res)


In [18]:
review = eval_df.iloc[27,0]

In [ ]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def predictor(texts):
    predictions = model.predict(texts)
    x = np.array(list(predictions)[1])
    return np.apply_along_axis(softmax, 1, x)


explainer = lime.lime_text.LimeTextExplainer(class_names=["0", "1"], bow=False)

str_to_predict = review
exp = explainer.explain_instance(str_to_predict, predictor, num_features=10, num_samples=200, top_labels=2)
exp.show_in_notebook(text=str_to_predict)

In [ ]:
exp.as_list()

In [ ]:
results = (model.predict(review))
print (results[1][0])
print(testset.iloc[27,0])

In [ ]:
%matplotlib inline
fig = exp.as_pyplot_figure()